In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import seaborn as sns

In [ ]:
from tensorflow import keras
from keras import Input, Sequential, Model
from keras.regularizers import l2
from keras.layers import Dense, Flatten, InputLayer, Reshape, BatchNormalization, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model, to_categorical

%matplotlib inline

In [ ]:
categories = {'cloudy': 0, 'desert': 1, 'green_area': 2, 'water': 3}

In [ ]:
def load_data(imgFolder, imgSize = (128, 128), scale = False):
  imgPath = []
  for dirName, _, fileNames in os.walk(imgFolder):
    for fileName in fileNames:
      imgPath.append(os.path.join(dirName, fileName))
  
  print("There are {} images in {}".format(len(imgPath), imgFolder))

  images = []
  labels = []

  for path in tqdm(imgPath):
    img = cv2.imread(path)
    img = cv2.resize(img, imgSize)
    img = np.array(img)

    images.append(img)
    labels.append(categories[path.split('/')[-2]])

  images = np.array(images)
  images = images.astype(np.int64)

  if scale:
    images = images / 255
  
  return imgPath, images, np.asarray(labels)

In [ ]:
imgSize = (128, 128)
imgFolder = os.path.join('/', 'kaggle', 'input', 'satellite-image-classification')
imgPath, images, labels = load_data(imgFolder, imgSize = imgSize)

images.shape

In [ ]:
plt.figure(figsize = (10, 10))
randInds = np.random.choice(len(imgPath), 49)
for i in range(49):
  plt.subplot(7, 7, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  imgInd = randInds[i]

  plt.imshow(np.squeeze(images[imgInd]), cmap = plt.cm.binary)

  label = list(categories.keys())[list(categories.values()).index(labels[imgInd])]

  plt.title(label)

In [ ]:
labels_df = pd.DataFrame(labels)
labels_df.value_counts()

In [ ]:
def build_model():
    cnnModel = Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = images.shape[1:]),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),

        Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),
        
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
        MaxPooling2D(2, 2),
        BatchNormalization(),
        Dropout(0.4),
        
        Flatten(),
        
        Dense(units = len(categories), activation = 'softmax')        
    ])
    
    return cnnModel

In [ ]:
model = build_model()
model.predict(images[[0]])
print(model.summary())

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [ ]:
labEnc = LabelEncoder()
labels = labEnc.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
labels[4567:4600]

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# images.shape
labels.shape

In [ ]:
from sklearn.model_selection import train_test_split as tts

xTrain, xTest, yTrain, yTest = tts(images, labels, test_size = 0.30, random_state = 42)

xTest, xVal, yTest, yVal = tts(xTest, yTest, test_size = 0.5, random_state = 42)

In [ ]:
print('Train:')
print(xTrain.shape, yTrain.shape)
print()
print('Test:')
print(xTest.shape, yTest.shape)
print()
print('Validation:')
print(xVal.shape, yVal.shape)

In [ ]:
history = model.fit(xTrain, yTrain,
                   batch_size = 37,
                   epochs = 8,
                   verbose = 1,
                   validation_data = (xVal, yVal))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc = 'upper right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc = 'upper right')
plt.show()

In [ ]:
def make_predictions(img):
    img = img.reshape(1, 128, 128, 3)
    
    predic = model.predict(img)
    truePredic = [tf.argmax(pred) for pred in predic]
    truePredic = np.array(truePredic)
    
    return list(categories.keys())[list(categories.values()).index(truePredic)]

In [ ]:
make_predictions(xTest[0])

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

In [ ]:
yPredd = [np.argmax(y) for y in yPred]
yTestt = [np.argmax(y) for y in yTest]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(yTestt, yPredd))

In [ ]:
categories.keys()

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score
from mlxtend.plotting import plot_confusion_matrix

cm = confusion_matrix(yTestt, yPredd)
plot_confusion_matrix(conf_mat = cm,
           figsize = (8, 7),
           class_names = categories.keys(),
           show_normed = True)

In [ ]:
plt.figure(figsize = (10, 10))
randInds = np.random.choice(xTest.shape[0], 49)

for i in range(49):
    plt.subplot(7, 7, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    imgInd = randInds[i]
    
    plt.imshow(np.squeeze(xTest[imgInd]), cmap = plt.cm.binary)
    
    label = make_predictions(xTest[imgInd])
    plt.xlabel(label)
    

In [ ]:
def evaluate_model(model):
    yPred = model.predict(xTest)
    yPredd = []
    yTestt = []
    
    for i in range(len(yPred)):
        yPredd.append(np.argmax(yPred[i]))
        yTestt.append(np.argmax(yTest[i]))
        
        print(classification_report(yTestt, yPredd))
        
        confMat = confusion_matrix(yTestt, yPredd) / np.sum(confusion_matrix(yTestt, yPredd))
        accuracy = precision_score(yTestt, yPredd, average = None)
        
        categList = ['cloudy', 'desert', 'green_area', 'water']
        
        print('Confusion Matrix:\n')
        print(confMat)
        
        hm = sns.heatmap(confMat, annot = True, fmt = '.2%', lw = 0.1, cmap = 'Blues', cbar = False)
        
        hm.set_title('Confusion Matrix\n')
        hm.set_xlabel('\nPredicted Classes')
        hm.set_ylabel('True Classes')
#         hm.xaxis.set_ticklabels(['cloudy', 'desert', 'green_area', 'water'])
#         hm.yaxis.set_ticklabels(['cloudy', 'desert', 'green_area', 'water'])
        
        plt.show()
        
        print('Accuracy For Each Class:\n')
        for i in range(len(accuracy)):
            print(f"{categList[i]}: {format(accuracy[i], '.2f')}")

In [ ]:
evaluate_model(model)

In [ ]:
model.save('satelliteImageClassifier.h5')